# Projet LLM - IA conversationnelle spécialisée dans les films
## De la récupération de texte à la création d’un modèle génératif (Python)

### 2.1

Films (recommandations/conseils, listing etc.)

Une IA conversationnel spécialisé dans les films, capable de :
– répondre à des questions sur les acteurs et films (métadonnées IMDb),
– suggérer des films selon les envies de l’utilisateur (genre, ambiance, popularité),
– générer des synopsis courts ou des pitchs de films,
– fournir la note IMDb et le nombre de votes.

### 2.2

Le système s’appuie sur les datasets IMDb non-commerciaux et sur un modèle distilgpt2 fine-tuné sur un corpus textuel cinéma construit à partir de ces mêmes données, afin de générer des réponses naturelles et contextualisées.


### 3.1 Récupération des données IMDb

In [7]:
import pandas as pd
import os

DATA_DIR = "./datas" 

def load_imdb_tsv(filename):
    filepath = os.path.join(DATA_DIR, filename)
    return pd.read_csv(filepath, sep='\t', dtype=str, na_values='\\N')

titles = load_imdb_tsv('title.basics.tsv.gz')
ratings = load_imdb_tsv('title.ratings.tsv.gz')
principals = load_imdb_tsv('title.principals.tsv.gz')
names = load_imdb_tsv('name.basics.tsv.gz')
akas = load_imdb_tsv('title.akas.tsv.gz')  

print(f"Titres: {len(titles)} lignes")
print(f"Ratings: {len(ratings)} lignes")
print(f"Principals: {len(principals)} lignes")
print(f"Noms: {len(names)} lignes")
print(f"Akas: {len(akas)} lignes")

print("\n=== TITLES ===")
print(titles.head())
print(f"Colonnes: {titles.columns.tolist()}")

print("\n=== RATINGS ===")
print(ratings.head())

print("\n=== PRINCIPALS ===")
print(principals.head())

print("\n=== NAMES ===")
print(names.head())


Titres: 12108232 lignes
Ratings: 1607015 lignes
Principals: 96248652 lignes
Noms: 14912263 lignes
Akas: 54070383 lignes

=== TITLES ===
      tconst titleType            primaryTitle           originalTitle  \
0  tt0000001     short              Carmencita              Carmencita   
1  tt0000002     short  Le clown et ses chiens  Le clown et ses chiens   
2  tt0000003     short            Poor Pierrot          Pauvre Pierrot   
3  tt0000004     short             Un bon bock             Un bon bock   
4  tt0000005     short        Blacksmith Scene        Blacksmith Scene   

  isAdult startYear endYear runtimeMinutes                    genres  
0       0      1894     NaN              1         Documentary,Short  
1       0      1892     NaN              5           Animation,Short  
2       0      1892     NaN              5  Animation,Comedy,Romance  
3       0      1892     NaN             12           Animation,Short  
4       0      1893     NaN              1                     S

### 3.2 Filtrer les films (exclure séries, documentaires, courts-métrages etc.)

Les datasets que l'on a récupéré sont extrêmement riches et lourds. Pour se concentrer sur les films "classiques", on va filtrer les titres selon plusieurs critères :
- Exclure les titres dont le type n'est pas "movie" (ex : "tvSeries", "short", "documentary" etc.)
- Garder uniquement les films sortis après l'an 2000 (pour se concentrer sur des films récents)
- Exclure les films avec une note IMDb inférieure à 5.0 (pour se concentrer sur des films appréciés)

In [8]:
import pandas as pd

# Keep only movies
movies = titles[titles['titleType'] == 'movie'].copy()
print(f"Rows after filtering rows to only get movies: {len(movies)} rows")

# Keep only movies from 1980 to 2024
movies = movies[(movies['startYear'] >= '1980') & (movies['startYear'] <= '2024')]
print(f"Rows after filtering movies (1980-2024): {len(movies)} rows")

# Keep only movies with a known runtime
movies['startYear'] = movies['startYear'].astype(int)
movies['runtimeMinutes'] = pd.to_numeric(movies['runtimeMinutes'], errors='coerce')

# Keep only movies longer than 45 minutes
movies = movies[movies['runtimeMinutes'] > 45]
print(f"Rows after filtering runtime > 45 min: {len(movies)} rows")

# Merge the movies dataset with ratings 
movies = movies.merge(ratings, on='tconst', how='inner')
print(f"Rows after ratings merge: {len(movies)} rows")

# Keep only movies with averageRating >= 5.0
movies = movies[movies['numVotes'].astype(int) >= 5000]
print(f"Rows after filtering numVotes >= 5000: {len(movies)} rows")

movies['averageRating'] = pd.to_numeric(movies['averageRating'], errors='coerce')
movies['numVotes'] = pd.to_numeric(movies['numVotes'], errors='coerce')

Rows after filtering rows to only get movies: 733058 rows
Rows after filtering movies (1980-2024): 437300 rows
Rows after filtering runtime > 45 min: 330537 rows
Rows after ratings merge: 222344 rows
Rows after filtering numVotes >= 5000: 15575 rows


### 3.3 Ajouter les Acteurs Principaux des Films

On va ajouter à notre dataset les acteurs principaux (top 3) pour chaque film, en utilisant le dataset `title.principals.tsv.gz`. Cela permettra au modèle d'améliorer la qualité des recommandations et des réponses générées, en liant certains films de styles différents à travers les différentes acteurs.

In [9]:

# Keep the 5 principal actors for each film
principals['ordering'] = pd.to_numeric(principals['ordering'], errors='coerce')
actors_in_films = principals[
    (principals['category'].isin(['actor', 'actress'])) &
    (principals['ordering'] <= 5)  
].copy()

actors_in_films = actors_in_films.merge(
    names[['nconst', 'primaryName']], 
    on='nconst', 
    how='left'
)

# Regroup actors by film
actors_by_film = actors_in_films.groupby('tconst')['primaryName'].apply(
    lambda x: ', '.join(x.dropna().unique())
).reset_index()
actors_by_film.columns = ['tconst', 'actors']

movies = movies.merge(actors_by_film, on='tconst', how='left')
movies['actors'] = movies['actors'].fillna('Unknown')

print(movies[['tconst', 'primaryTitle', 'startYear', 'genres', 'actors', 'averageRating', 'numVotes']].head(10))

movies.to_parquet('movies_filtered.parquet', index=False)
movies.to_csv('movies_filtered.csv', index=False)

print("Saved")

      tconst                      primaryTitle  startYear  \
0  tt0035423                    Kate & Leopold       2001   
1  tt0069049        The Other Side of the Wind       2018   
2  tt0076276         Who's Singin' Over There?       1980   
3  tt0078813                         The Miser       1980   
4  tt0078935                Cannibal Holocaust       1980   
5  tt0079285                          Saturn 3       1980   
6  tt0079579  Moscow Does Not Believe in Tears       1980   
7  tt0079788                  Zombie Holocaust       1980   
8  tt0079820      The King and the Mockingbird       1980   
9  tt0079891                The Shaolin Temple       1982   

                     genres  \
0    Comedy,Fantasy,Romance   
1                     Drama   
2    Adventure,Comedy,Drama   
3                    Comedy   
4          Adventure,Horror   
5   Adventure,Horror,Sci-Fi   
6      Comedy,Drama,Romance   
7  Adventure,Horror,Mystery   
8  Animation,Family,Fantasy   
9       Action,Com

### 4. Génération du texte

In [10]:
import pandas as pd
from tqdm import tqdm
import random

movies = pd.read_parquet('movies_filtered.parquet')
corpus_texts = []

templates = [
    # --- without scores ---
    "{title} est un film {genres} sorti en {year}, porté par {actors}. Il mise surtout sur l’ambiance et le récit pour immerger le spectateur.\n\n",
    "Sorti en {year}, {title} est un long-métrage {genres} avec {actors}. Le film se distingue par son ton particulier et sa mise en scène.\n\n",
    "{title} ({year}) est un film {genres}. Avec {actors}, il propose une histoire qui marie émotions et divertissement.\n\n",
    "Dans {title}, sorti en {year}, {actors} incarnent des personnages marquants. Ce film {genres} s’adresse surtout à ceux qui aiment les univers travaillés.\n\n",
    "{title} est une production {genres} de {year}. Le casting, mené par {actors}, donne une identité forte au film.\n\n",
    "{title}, sorti en {year}, appartient au genre {genres}. Il s’appuie sur {actors} pour donner vie à son récit.\n\n",
    "Avec {actors} au casting, {title} propose une expérience {genres} sortie en {year}, centrée sur ses personnages et son atmosphère.\n\n",

    # --- quality scores ---
    "{title} est un film {genres} sorti en {year} avec {actors}. Il est généralement {appreciation}.\n\n",
    "Avec {actors} en tête d’affiche, {title} propose un récit {genres} sorti en {year}. Le film est considéré comme {appreciation}.\n\n",
    "{title} ({year}) appartient au genre {genres}. Grâce à {actors}, le film a laissé une impression {appreciation} sur son public.\n\n",
    "Parmi les films {genres} sortis en {year}, {title} se distingue par son casting ({actors}) et un accueil {appreciation}.\n\n",
    "{title} est souvent cité comme un exemple {genres} {appreciation}, notamment grâce à la performance de {actors}.\n\n",

    # --- with score ---
    "{title} est un film {genres} sorti en {year} avec {actors}. Sur IMDb, il bénéficie d’une note d’environ {rating}/10, signe d’un intérêt réel du public.\n\n",
    "Film {genres} de {year}, {title} réunit {actors}. Sa note autour de {rating}/10 sur IMDb reflète des retours globalement positifs.\n\n",
    "{title} ({year}) met en avant {actors} dans un récit {genres}. La communauté IMDb lui attribue une note proche de {rating}/10.\n\n",
    "{title} est considéré comme un film {genres} solide. Sorti en {year} avec {actors}, il est évalué à environ {rating}/10 par les utilisateurs d’IMDb.\n\n",
    "Parmi les films {genres}, {title} ({year}) avec {actors} obtient une note avoisinant {rating}/10 sur IMDb, ce qui traduit son accueil.\n\n",
    "{title}, film {genres} sorti en {year}, met en scène {actors}. Sa note sur IMDb tourne autour de {rating}/10, ce qui reste cohérent avec les avis du public.\n\n",

    # --- Recommendation ---
    "Si tu cherches un film {genres} sorti en {year}, {title} avec {actors} est une option intéressante à considérer.\n\n",
    "Pour une soirée {genres}, {title} ({year}) avec {actors} peut être un bon choix, souvent {appreciation} par ceux qui l’ont vu.\n\n",
]


def rating_to_appreciation(rating):
    try:
        r = float(rating)
    except (TypeError, ValueError):
        return "accueilli de manière mitigée"
    if r >= 8.0:
        return "très apprécié du public"
    elif r >= 7.0:
        return "bien accueilli par les spectateurs"
    elif r >= 6.0:
        return "reçu de façon mitigée mais intéressant pour certains"
    else:
        return "plutôt mal reçu par le public"

def format_genres(genres_str):
    if pd.isna(genres_str) or genres_str == 'Unknown':
        return 'inconnu'
    genres = genres_str.split(',')
    if len(genres) == 1:
        return f"de {genres[0].lower()}"
    elif len(genres) == 2:
        return f"de {genres[0].lower()} et de {genres[1].lower()}"
    else:
        return f"de {', '.join(g.lower() for g in genres[:-1])} et de {genres[-1].lower()}"


def format_actors(actors_str, max_actors=3):
    if pd.isna(actors_str) or actors_str == 'Unknown':
        return "des acteurs inconnus"
    actors = actors_str.split(', ')
    if len(actors) > max_actors:
        actors = actors[:max_actors]
    if len(actors) == 1:
        return actors[0]
    elif len(actors) == 2:
        return f"{actors[0]} et {actors[1]}"
    else:
        return f"{', '.join(actors[:-1])} et {actors[-1]}"
    

for idx, row in tqdm(movies.iterrows(), total=len(movies)):
    title = row['primaryTitle']
    year = row['startYear']
    genres = format_genres(row['genres'])
    actors = format_actors(row['actors'])
    rating = row['averageRating']
    votes = row['numVotes']
    appreciation = rating_to_appreciation(rating)

    template = random.choice(templates)

    text = template.format(
        title=title,
        year=year,
        genres=genres,
        actors=actors,
        rating=rating,
        votes=f"{votes:,}",
        appreciation=appreciation,
    )
    corpus_texts.append(text)


print(f"{len(corpus_texts)} texts")

import pickle
with open('corpus_texts.pkl', 'wb') as f:
    pickle.dump(corpus_texts, f)

with open('corpus_sample.txt', 'w', encoding='utf-8') as f:
    f.write("\n".join(corpus_texts[:100]))  



100%|██████████| 15575/15575 [00:00<00:00, 19420.20it/s]

15575 texts


5. Creation du dataset pour le fine-tuning

In [11]:
import re
from datasets import Dataset

with open('corpus_texts.pkl', 'rb') as f:
    corpus_texts = pickle.load(f)

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = text.replace('\xa0', ' ')
    return text.strip()

cleaned_texts = [clean_text(t) for t in corpus_texts]

dataset = Dataset.from_dict({"text": cleaned_texts})

print(f"dataset size : {len(dataset)}")

dataset.save_to_disk('imdb_corpus_dataset')
print("saved")

c:\Users\Frédéric\Developer\4iabd_t1_llm_project\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


dataset size : 15575


Saving the dataset (1/1 shards): 100%|██████████| 15575/15575 [00:00<00:00, 1639181.11 examples/s]

saved


6. Choisir un modèle de base : distilgpt2

In [12]:
import torch
from datasets import Dataset, load_from_disk
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    Trainer, 
    TrainingArguments,
    DataCollatorForLanguageModeling
)

dataset = load_from_disk('imdb_corpus_dataset')
model_name = "distilgpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


7. Tokenisation du dataset

In [13]:

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

num_params = sum(p.numel() for p in model.parameters())
block_size = 128

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=block_size,
    )

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"],
    desc="Tokenisation"
)

print(f"Tokenized {len(tokenized_dataset)} items")

tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["test"]

Tokenisation: 100%|██████████| 15575/15575 [00:01<00:00, 13590.75 examples/s]

Tokenized 15575 items


### 8. Prepare Fine-tuning with Trainer

In [14]:

output_dir = "./distilgpt2-imdb-finetuned"

training_args = TrainingArguments(
    output_dir=output_dir,
    eval_strategy="epoch",  
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

# 8.1

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

# 8.2

trainer.train()



c:\Users\Frédéric\Developer\4iabd_t1_llm_project\env\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

### 9. Sauvegarder le modèle fine-tuné

In [ ]:

trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

eval_results = trainer.evaluate()
print(eval_results)

c:\Users\Frédéric\Developer\4iabd_t1_llm_project\env\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 1.6746909618377686, 'eval_runtime': 122.1659, 'eval_samples_per_second': 12.753, 'eval_steps_per_second': 3.192, 'epoch': 3.0}


### 10. Générer du texte avec le modèle  

In [ ]:
from transformers import pipeline
import torch


model_path = "./distilgpt2-imdb-finetuned"

generator = pipeline(
    "text-generation",
    model=model_path,
    tokenizer=model_path,
    torch_dtype=torch.float16,
    device=0 if torch.cuda.is_available() else -1  
)

def generate_response(prompt, max_length=150, num_sequences=3):
    print(f"\n{'='*60}")
    print(f"PROMPT: {prompt}")
    print(f"{'='*60}")
    
    outputs = generator(
        prompt,
        max_length=max_length,
        num_return_sequences=num_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.8,
        repetition_penalty=1.2,
        pad_token_id=generator.tokenizer.eos_token_id,
    )
    
    for i, out in enumerate(outputs, 1):
        generated_text = out["generated_text"][len(prompt):].strip()
        print(f"\n--- Génération {i} ---")
        print(generated_text)
    
    return outputs

generate_response(
    "Propose-moi un film d'action récent avec de bons effets spéciaux.",
    max_length=150
)

generate_response(
    "Je veux un film avec Tom Cruise. Que me conseilles-tu ?",
    max_length=150
)

generate_response(
    "Donne-moi un court synopsis d'un film de science-fiction sorti dans les années 2000.",
    max_length=200
)

generate_response(
    "Quelle est la note du film Inception et pourquoi est-il si populaire ?",
    max_length=150
)

generate_response(
    "J'ai envie d'un film drôle et intelligent. Une idée ?",
    max_length=150
)

generate_response(
    "Liste les acteurs principaux du film Titanic.",
    max_length=150
)


`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



PROMPT: Propose-moi un film d'action récent avec de bons effets spéciaux.


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



--- Génération 1 ---
Ce film de drama et est un film de thriller sorti en 2011, il obtient une note of 6.6/10 sur IMDb. Avec des acteurs inconnus, il obtient une noteandie note 7.2/10 sur IMDb. Note IMDb: 5.8/10 (5,955 votes). Score: 3.7/10 sur IMDb. Note IMDb: 4.3/10 sur IMDb. Note IMDb: 6.0/10 sur IMDb. Note IMDb.: 5.4/10 sur IMDb. Note IMDb 20/10 sur IMDb. Note IMDb: 6.1/10 sur IMDb. Note IMDb: 5.9/10 sur IMB. Note IMDb: 5.3/10 sur IMDb. Note IMDb: 4.9 in 7.6/10 sur IMDb. Note IMDb: 5.6/10 sur IMDb. Note IMDb 8 2.1/10 sur IMDb. Note IMDb: 3.2/11 sur IMDb. Note IMDb: 4.4/12 Surim et 1.8/10 sur IMDb. Note IMDb: 3

--- Génération 2 ---
Ce film de action, comedy et est un film de crime sorti en 2023. Avec Nicolas Cage, Julianne Moore et Dermot Mulroney, il obtient une note of 6.6/10 sur IMDb.3/10 (8,711 votes). Score: 7.4/10 sur IMDb.1/10 (15,941 votes). Score: 8.0/10 sur IMDb.2/10 (5,948 votes). Score: 5.1/10 sur IMDb.1/10 (12,822 votes). Note IMDb: 4.8/10 sur IMDb.1/10 (21,644 votes)

Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



--- Génération 1 ---
Avec Nicolas Cage, Jai Courtney Scott Thomas Jr. et Elizabeth Banks.. Note IMDb: 6.5/10 (48,945 votes). IMDb: 5.7/10 (23,859 votes). Score: 7.1/10 (21,552 votes). IMDb de action, adventure et une note de thriller a il obtient une note of 5.3/10 sur IMDb. Score: 4.6/10 (26,084 votes). Score: 3.3/10 (16,844 votes). IMDb: 5.0/10 (14,879 votes) -2.5/10 sur IMDb: 5.2/10 sur IMDb.: 4.7/10 sur IMDb: 5.7/10 sur IMDb: 4.3/10 sur IMDb: 5.5/10 sur IMDband Quelle: 4.8/10 sur ImDband 6.3x IMDb(8,963 votes). IMDb: 5.0/10 sur IMDbendiève des acteurs inconnus. IMDb: 5.6/10 sur IMDbst il obtient une note en 8.

--- Génération 2 ---
Avec Philippe Nivet, Mélanie Laurent et Jean Le Bourgne. Ce film de drama et une thriller sorti en 2020 il obtient une note of 7.1/10 sur IMDb.7/10 sur IMDb.6/10 sur IMDb.8/10 sur 6.5/10 sur IMDb.3/10 sur 6.6/10 sur IMDb.4 in 8.0/10 sur IMDb.2/10 sur 5.9/10 sur IMDb.3/10 sur 5.9/10 sur 4.1/10 sur 5.2/10 sur 3.4/10 sur 2.2/10 sur 1.9/10 sur 1.6/10 sur 1.

Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



--- Génération 1 ---
une note de 7.9 sur IMDb. Avec Jang Dong-ho, Park Yoon-jae et Yeong Seo-joo. Score: 6.5/10.7/10. Note IMDb: 8.1/10 (8,904 votes). Score of 4.5/10 sur IMDb. Note Tang Hwan Kim et Jung Hyuk-soo. IMDb: 5.6/10 (12,828 votes). IMDb: 3.3/10 (19,848 votes). Note est un film de comedy, drama et en romance sorti en 2022. Score: 2.0/10 sur IMDb. Note IMDb.: 1.9/10 sur IMDb. Note IMDb: 7.2/10 sur IMDb. Note IMDb: 7.4/11 sur IMDb. Note IMDb: 6.4/10 sur IMDb. Note IMDb avec Kang Hee-ju, Kim Soo-jung et Kim Byung-won. IMDb: 6.4/10 sur IMDb. Note IMB: 6.9/10 sur IMDb. Note IMDb: 6.8/10 sur IMDb.

--- Génération 2 ---
une note de 6.7/10 sur IMDb. Thiruvothu, Anjali D'Cruz et Pankaj Tripathi Vijayakonda. Ce film de drama et est un film de musical sorti en 2013, il obtient une note de 7.4/10 sur IMDb. Avec Shraddha Roshanupadhyay Jr., il obtient une note of 8.2/10 sur IMDb. Note IMDb: 5.7/10 sur IMDb. Note IMDb: 4.8/10 sur IMDb. Note IMDb: 1.9/10 sur IMDb. Note IMDb: 3.0/10 sur IMD

[{'generated_text': 'Liste les acteurs principaux du film Titanic. Ce film de documentary et est un film sorti en 2023, il obtient une note des 7.3/10 sur IMDb. Avec des acteurs inconnus, il obtient une 6.6/10 sur IMDb. Anissa Bledelogui, Antonio Rochaazquez Jr., and Anthony Anderson-Staunton Cossicko: 4.9/10 sur IMDb. Score: 8.2/10 sur IMDb. Note IMDb: 1.9/10 sur IMDb. Score: 5.8/10 sur IMDb. Note IMDb.: 3.1/10 sur IMDb. Note IMDb: 7.7/10 sur IMDb. Note IMDb!: 2.0/10 sur IMDb. Note IMDb (11,743 votes). IMDb: 7.5/10 sur IMDb. Note IMDb: 5.4/10 sur IMDb. Note IMDb: 5.2/20 sur IMandation: 6.1/10 sur IMDb. Note IMDb: 6.4/10 sur Imandation of Interest: 7.5 Surimed: 7.7/10 sur IMDb. Note'},
 {'generated_text': 'Liste les acteurs principaux du film Titanic. Ce film de drama et est un film de thriller sorti en 2017, il obtient une note IMDb: 5.6/10 (7,907 votes). Avec des acteurs inconnus, il obtient une note of 7.3/10 sur IMDb. Score: 6.1/10 sur IMDb. Note IMDb.: 4.8/10 (5,038 votes). Score:

### 11. Evaluation simple du modèle génératif

- [ ] Cohérence : le texte est-il logique, grammatical ?
->

- [ ] Style : ressemble-t-il aux textes d’entraînement ?
->

- [ ] Spécialisation : le modèle reste-t-il dans le bon domaine (cuisine, IT, etc.) ?
->

- [ ] Hallucinations : invente-t-il des choses fausses / dangereuses ?
->

### 12. Extensions possibles

#### 12.1 Alpaca 

In [ ]:
import pandas as pd
import random
import json

movies = pd.read_parquet("movies_filtered.parquet")

instructions = []

def pick_random_movie(df):
    return df.sample(1).iloc[0]

for _ in range(5000):
    row = pick_random_movie(movies)
    title = row["primaryTitle"]
    year = row["startYear"]
    genres = row["genres"]
    actors = row["actors"]
    rating = row["averageRating"]
    votes = row["numVotes"]
    appreciation = rating_to_appreciation(rating)

    itype = random.choice(["reco", "resume", "actors", "popularity", "vibe"])

    if itype == "reco":
        instruction = random.choice([
            f"Recommande-moi un bon film {genres.lower()} à regarder ce soir.",
            f"Je cherche un film {genres.lower()} récent à voir. Tu me proposes quoi ?",
            f"Si j’aime les films {genres.lower()}, quel film me conseilles-tu ?",
        ])
        output = (
            f"Tu peux regarder {title} ({year}). "
            f"C’est un film {genres.lower()} avec {actors}. "
            f"Il est {appreciation}."
        )

    elif itype == "resume":
        instruction = random.choice([
            f"Donne un court descriptif d’un film {genres.lower()} sorti en {year}.",
            f"Résume brièvement un film {genres.lower()} avec {actors}.",
            f"Présente en quelques phrases un film {genres.lower()} marquant des années {year}.",
        ])
        output = (
            f"{title} est un film {genres.lower()} sorti en {year} avec {actors}. "
            f"Il raconte une histoire typique de ce genre et est {appreciation}."
        )

    elif itype == "actors":
        instruction = random.choice([
            f"Quels sont les acteurs principaux du film {title} ?",
            f"Qui joue dans le film {title} ({year}) ?",
            f"Donne-moi les acteurs principaux de {title}.",
        ])
        output = f"Les principaux acteurs de {title} ({year}) sont {actors}."

    elif itype == "popularity":
        instruction = random.choice([
            f"Explique pourquoi un film comme {title} est connu du grand public.",
            f"Pourquoi {title} est-il autant cité parmi les films {genres.lower()} ?",
            f"Qu’est-ce qui peut expliquer le succès de {title} ({year}) ?",
        ])
        output = (
            f"{title} ({year}) est un film {genres.lower()} avec {actors}. "
            f"Il est {appreciation}, ce qui explique qu’il soit souvent recommandé."
        )

    else:  # vibe
        instruction = random.choice([
            f"Décris l’ambiance générale d’un film {genres.lower()} comme {title}.",
            f"Quel type d’ambiance peut-on attendre d’un film {genres.lower()} tel que {title} ?",
            f"Parle-moi de l’atmosphère d’un film {genres.lower()} sorti en {year}.",
        ])
        output = (
            f"{title} est un film {genres.lower()} sorti en {year} avec {actors}. "
            f"L’ambiance correspond bien à ce qu’on attend d’un film de ce genre."
        )


    instructions.append(
        {
            "instruction": instruction,
            "input": "",
            "output": output,
        }
    )

with open("imdb_instructions.jsonl", "w", encoding="utf-8") as f:
    for ex in instructions:
        f.write(json.dumps(ex, ensure_ascii=False) + "\n")

print(f"Dataset d’instructions créé: {len(instructions)} exemples")

# ---------

from datasets import load_dataset

dataset = load_dataset("json", data_files="imdb_instructions.jsonl")["train"]

def format_example(example):
    return (
        f"Instruction: {example['instruction']}\n"
        f"Réponse: {example['output']}\n\n"
    )

formatted_texts = [format_example(ex) for ex in dataset]

from datasets import Dataset
hf_dataset = Dataset.from_dict({"text": formatted_texts})
hf_dataset.save_to_disk("imdb_instructions_dataset")
print(hf_dataset[0]["text"])

# Go to Step 6 to fine-tune the model with this dataset.


✓ Dataset d’instructions créé: 5000 exemples


Generating train split: 5000 examples [00:00, 331817.35 examples/s]
Saving the dataset (1/1 shards): 100%|██████████| 5000/5000 [00:00<00:00, 1619422.39 examples/s]

Instruction: Quels sont les acteurs principaux du film The Last Days of Disco ?
Réponse: Les principaux acteurs de The Last Days of Disco (1998) sont Chloë Sevigny, Kate Beckinsale, Chris Eigeman, Mackenzie Astin, Matt Keeslar.




In [ ]:
### IGNORE THIS LAZY COPY PAST FROM 6-11

import torch
from datasets import Dataset, load_from_disk
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    Trainer, 
    TrainingArguments,
    DataCollatorForLanguageModeling
)

dataset = load_from_disk('imdb_instructions_dataset')
model_name = "distilgpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

num_params = sum(p.numel() for p in model.parameters())
block_size = 128

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=block_size,
    )

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"],
    desc="Tokenisation"
)

print(f"Tokenized {len(tokenized_dataset)} items")

tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["test"]


output_dir = "./distilgpt2-imdb-instructions-finetuned"

training_args = TrainingArguments(
    output_dir=output_dir,
    eval_strategy="epoch",  
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

trainer.train()


trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

eval_results = trainer.evaluate()
print(eval_results)

Tokenisation: 100%|██████████| 5000/5000 [00:00<00:00, 11166.50 examples/s]


Tokenized 5000 items


c:\Users\Frédéric\Developer\4iabd_t1_llm_project\env\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,1.530300,1.490783
2,1.327200,1.451694
3,1.337700,1.444844


c:\Users\Frédéric\Developer\4iabd_t1_llm_project\env\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\Frédéric\Developer\4iabd_t1_llm_project\env\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
There were missing keys in the checkpoint model loaded: ['lm_head.weight'].
c:\Users\Frédéric\Developer\4iabd_t1_llm_project\env\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 1.4448442459106445, 'eval_runtime': 38.5274, 'eval_samples_per_second': 12.978, 'eval_steps_per_second': 3.244, 'epoch': 3.0}


In [ ]:

model_path = "./distilgpt2-imdb-instructions-finetuned"

generator = pipeline(
    "text-generation",
    model=model_path,
    tokenizer=model_path,
    torch_dtype=torch.float16,
    device=0 if torch.cuda.is_available() else -1  
)

def generate_response(prompt, max_length=150, num_sequences=3):
    print(f"\n{'='*60}")
    print(f"PROMPT: {prompt}")
    print(f"{'='*60}")
    
    outputs = generator(
        prompt,
        max_length=max_length,
        num_return_sequences=num_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.8,
        repetition_penalty=1.2,
        pad_token_id=generator.tokenizer.eos_token_id,
    )
    
    for i, out in enumerate(outputs, 1):
        generated_text = out["generated_text"][len(prompt):].strip()
        print(f"\n--- Génération {i} ---")
        print(generated_text)
    
    return outputs

generate_response(
    "Propose-moi un film d'action récent avec de bons effets spéciaux.",
    max_length=150
)

generate_response(
    "Je veux un film avec Tom Cruise. Que me conseilles-tu ?",
    max_length=150
)

generate_response(
    "Donne-moi un court synopsis d'un film de science-fiction sorti dans les années 2000.",
    max_length=200
)

generate_response(
    "Quelle est la note du film Inception et pourquoi est-il si populaire ?",
    max_length=150
)

generate_response(
    "J'ai envie d'un film drôle et intelligent. Une idée ?",
    max_length=150
)

generate_response(
    "Liste les acteurs principaux du film Titanic.",
    max_length=150
)


HFValidationError: Repo id must use alphanumeric chars, '-', '_' or '.'. The name cannot start or end with '-' or '.' and the maximum length is 96: './distilgpt2-imdb-instructions-finetune     d'.

### 12.2 LoRA (PEFT) 

In [ ]:
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType

dataset = load_from_disk("imdb_instructions_dataset")

model_name = "gpt2-medium"  
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["c_attn", "c_proj"], 
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()  

block_size = 256

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=block_size,
    )

tokenized = dataset.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized = tokenized.train_test_split(test_size=0.1, seed=42)
train_dataset = tokenized["train"]
eval_dataset = tokenized["test"]

training_args = TrainingArguments(
    output_dir="./gpt2-imdb-lora",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_steps=10,
)

from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

trainer.train()
trainer.save_model("./gpt2-imdb-lora")
tokenizer.save_pretrained("./gpt2-imdb-lora")


In [ ]:

model_path = "./gpt2-imdb-lora"

generator = pipeline(
    "text-generation",
    model=model_path,
    tokenizer=model_path,
    torch_dtype=torch.float16,
    device=0 if torch.cuda.is_available() else -1  
)

def generate_response(prompt, max_length=150, num_sequences=3):
    print(f"\n{'='*60}")
    print(f"PROMPT: {prompt}")
    print(f"{'='*60}")
    
    outputs = generator(
        prompt,
        max_length=max_length,
        num_return_sequences=num_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.8,
        repetition_penalty=1.2,
        pad_token_id=generator.tokenizer.eos_token_id,
    )
    
    for i, out in enumerate(outputs, 1):
        generated_text = out["generated_text"][len(prompt):].strip()
        print(f"\n--- Génération {i} ---")
        print(generated_text)
    
    return outputs

generate_response(
    "Propose-moi un film d'action récent avec de bons effets spéciaux.",
    max_length=150
)

generate_response(
    "Je veux un film avec Tom Cruise. Que me conseilles-tu ?",
    max_length=150
)

generate_response(
    "Donne-moi un court synopsis d'un film de science-fiction sorti dans les années 2000.",
    max_length=200
)

generate_response(
    "Quelle est la note du film Inception et pourquoi est-il si populaire ?",
    max_length=150
)

generate_response(
    "J'ai envie d'un film drôle et intelligent. Une idée ?",
    max_length=150
)

generate_response(
    "Liste les acteurs principaux du film Titanic.",
    max_length=150
)
